# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_info = pd.read_csv("../WeatherPy/weather_df.csv")
weather_info

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,39.99,66,95,5.99,US,1604378115
1,dikson,73.51,80.55,10.69,95,13,8.14,RU,1604378210
2,lexington,37.99,-84.48,36.00,55,1,6.93,US,1604378145
3,namatanai,-3.67,152.43,83.55,74,68,4.63,PG,1604378210
4,butaritari,3.07,172.79,80.85,76,1,17.05,KI,1604378210
...,...,...,...,...,...,...,...,...,...
534,okhotsk,59.38,143.30,31.59,94,100,23.73,RU,1604378357
535,dalbandin,28.89,64.41,69.55,15,0,5.35,PK,1604378358
536,suleja,9.18,7.18,69.80,83,0,4.70,NG,1604378358
537,paita,-5.09,-81.11,61.56,90,86,7.78,PE,1604378358


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Congiure gmaps
gmaps.configure(api_key=g_key)

#Store lat/lng in location
location = weather_info[["Lat", "Lng"]]

#Add humidity
humidity = weather_info["Humidity"]

In [4]:
#Add points to map
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
max_humidity = np.max(humidity)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Add heat layer
heat = gmaps.heatmap_layer(
    location, 
    weights = humidity, 
    dissipating = False,  
    point_radius=2
    )
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions.
vacation_df = weather_info.loc[(weather_info["Wind Speed"] <= 10) & 
                                (weather_info["Cloudiness"] == 0) & \
                                (weather_info["Max Temp"] >= 70) & 
                                (weather_info["Max Temp"] <= 80)].dropna()

vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,cayenne,4.93,-52.33,75.20,100,0,1.12,GF,1604378182
29,puerto narino,-3.77,-70.38,77.00,100,0,2.24,CO,1604378218
37,upington,-28.45,21.26,75.20,41,0,9.17,ZA,1604378220
58,batemans bay,-35.72,150.18,75.99,57,0,1.01,AU,1604378226
113,providencia,28.72,-111.58,73.99,40,0,1.99,MX,1604378009
220,flinders,-34.58,150.86,73.99,60,0,4.00,AU,1604378272
224,lazaro cardenas,17.96,-102.20,77.85,65,0,3.56,MX,1604378272
280,coahuayana,18.73,-103.68,75.88,71,0,2.84,MX,1604378287
308,tomatlan,19.93,-105.25,72.01,82,0,2.80,MX,1604378115
383,longhua,23.61,114.19,78.78,46,0,7.25,CN,1604378318


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacation_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,cayenne,GF,4.93,-52.33,
29,puerto narino,CO,-3.77,-70.38,
37,upington,ZA,-28.45,21.26,
58,batemans bay,AU,-35.72,150.18,
113,providencia,MX,28.72,-111.58,
220,flinders,AU,-34.58,150.86,
224,lazaro cardenas,MX,17.96,-102.20,
280,coahuayana,MX,18.73,-103.68,
308,tomatlan,MX,19.93,-105.25,
383,longhua,CN,23.61,114.19,


In [8]:
query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"

    response = requests.get(query_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print(f"error")

error
error
error


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,cayenne,GF,4.93,-52.33,Royal Amazonia
29,puerto narino,CO,-3.77,-70.38,Eco Hotel Aldea Amazonas
37,upington,ZA,-28.45,21.26,Allianto Boutique Hotel & Spa
58,batemans bay,AU,-35.72,150.18,Batemans Bay Hotel
113,providencia,MX,28.72,-111.58,
220,flinders,AU,-34.58,150.86,Eternity 141
224,lazaro cardenas,MX,17.96,-102.20,Hotel de Casablanca
280,coahuayana,MX,18.73,-103.68,Hotel Posada Navideña
308,tomatlan,MX,19.93,-105.25,Hotel Hacienda Vieja
383,longhua,CN,23.61,114.19,


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
plt.savefig("../VacationPy/Images/Heatmap_Anna.png")

<Figure size 432x288 with 0 Axes>